# Connecting to a port in the container

Another common use case is that we start some kind of long-running process in the container, and talk to it through a port. That process could be a Jupyter Notebook, for example. 

For demonstration purposes, we will use Python's in-built web server. To run it from the host, we could use

In [ ]:
!python -m http.server


Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...


If you need to use the Juptyter Notebook in a container, the best advice is to start your Dockerfile with `FROM jupyter/...`, and to make use of the Dockerfiles that the Jupyter team provides already (https://github.com/jupyter/docker-stacks).